In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pickle
import pandas as pd

In [4]:
with open('final.pkl', 'rb') as f:
    df = pickle.load(f)

In [5]:
df.head()

,username,eye_color,hair,skin_tone,skin_type,skin_concerns,foundation_color,review,recommend,unhelpful,helpful,star
0,creecreex,Brown,Brunette,Deep,Normal,Cellulite,370,I have 370 and I got matched at Sephora. I lik...,1,0,14,100
1,Jijine,Brown,Black,Dark,Combination,Unknown,470,It is like my skin but better. very lightweigh...,1,0,5,100
2,bloodyblondie,Blue,Black,Porcelain,Dry,Blackheads,100,I was so nervous about trying this product bec...,1,0,5,100
3,miralanani,Brown,Brunette,Medium,Combination,Acne,270,This product was amazing! Just fell in love! T...,1,0,5,100
4,shaaliyah,Brown,Black,Deep,Combination,Acne,370,Since purchasing this foundation it's been my ...,1,0,38,100


# Count Vectorize and Insert Stop Workfs

In [6]:
from gensim import corpora, models, similarities, matutils
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import logging
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [7]:
stop_words_list=list(ENGLISH_STOP_WORDS.copy())

In [8]:
new_words=['coverage', 'foundation', 'fenty', 'finish', 'does', 've', 'doesn', 'look', 'like', 'product', 'don', 'face', 'just', 'really']
stop_words_list.extend(new_words)

In [9]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),
                                   stop_words=stop_words_list, max_df=0.6, token_pattern="\\b[a-z][a-z]+\\b")

# Visualize CV DataFrame

In [11]:
cv_as_array= count_vectorizer.fit_transform(df['review']).toarray()

In [12]:
dt=pd.DataFrame(cv_as_array, columns=count_vectorizer.get_feature_names())

In [13]:
dt.shape

(2657, 42453)

In [28]:
dt

,aaannnddd,aaannnddd exactly,abh,abh contour,abh urban,able,able apply,able blend,able build,able buy,...,zone used,zone usually,zone wanted,zone wear,zone wears,zone whoa,zone work,zone worn,zones,zones stays
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Fit and Fit Transform

In [14]:
count_vectorizer.fit(df['review'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.6, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None,
        stop_words=['into', 'herein', 'so', 'upon', 'whereafter', 'than', 'yourself', 'ltd', 'towards', 'never', 'name', 'about', 'there', 'thick', 'anyway', 'above', 'both', 'hundred', 'were', 'any', 'cant', 'therefore', 'fill', 'me', 'nor', 'now', 'toward', 'as', 'give', 'those', 'thin', 'least', 'would',...fenty', 'finish', 'does', 've', 'doesn', 'look', 'like', 'product', 'don', 'face', 'just', 'really'],
        strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
        tokenizer=None, vocabulary=None)

In [10]:
cv_data=count_vectorizer.fit_transform(df['review'])

# Corpus

In [15]:
counts = count_vectorizer.transform(df['review']).transpose()

In [16]:
counts.shape

(42453, 2657)

In [17]:
corpus = matutils.Sparse2Corpus(counts)

In [18]:
id2word = { identifier: word for word, identifier in count_vectorizer.vocabulary_.items()}

In [19]:
len(id2word)

42453

# LDA

In [20]:
lda = models.LdaModel(corpus=corpus, num_topics=5, minimum_probability=0.03, id2word=id2word, passes=100)

In [21]:
lda.print_topics()

[(0,
  '0.011*"love" + 0.006*"shade" + 0.005*"oily" + 0.005*"day" + 0.005*"matte" + 0.004*"dry" + 0.004*"amazing" + 0.004*"great" + 0.004*"color" + 0.003*"perfect"'),
 (1,
  '0.009*"dry" + 0.006*"love" + 0.006*"oily" + 0.005*"primer" + 0.004*"day" + 0.004*"matte" + 0.003*"shade" + 0.003*"good" + 0.003*"looks" + 0.003*"tried"'),
 (2,
  '0.008*"love" + 0.006*"dry" + 0.004*"oily" + 0.004*"color" + 0.004*"shade" + 0.004*"good" + 0.003*"dry skin" + 0.003*"great" + 0.003*"matte" + 0.003*"primer"'),
 (3,
  '0.006*"love" + 0.005*"matte" + 0.005*"dry" + 0.005*"use" + 0.004*"oily" + 0.004*"color" + 0.004*"light" + 0.004*"perfect" + 0.004*"looks" + 0.004*"amazing"'),
 (4,
  '0.007*"love" + 0.006*"oily" + 0.005*"day" + 0.004*"shade" + 0.004*"great" + 0.004*"tried" + 0.003*"work" + 0.003*"looks" + 0.003*"perfect" + 0.003*"dry"')]

# LSA

In [22]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [23]:
from sklearn.decomposition import NMF, TruncatedSVD

In [24]:
n_comp = 5
lsa = TruncatedSVD(n_components=n_comp)

lsa_data = lsa.fit_transform(cv_data)

In [25]:
display_topics(lsa,count_vectorizer.get_feature_names(),10)


Topic  0
dry, love, oily, shade, matte, day, primer, color, tried, use

Topic  1
dry, dry skin, primer, patches, dry patches, good, looked, use, skin dry, combination

Topic  2
love, love love, dry, dry skin, oily, wanted love, oily skin, absolutely love, wanted, absolutely

Topic  3
oily, oily skin, day, hours, matte, end, end day, nose, skin oily, combination

Topic  4
color, tried, primer, looked, did, work, brush, didn, wanted, beauty


# Attempt to Visualize

In [26]:
import pyLDAvis
import pyLDAvis.sklearn

In [27]:
pyLDAvis.sklearn.prepare(lsa, cv_data, count_vectorizer)

/Users/mayamadhavan/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: invalid value encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/Users/mayamadhavan/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: invalid value encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/Users/mayamadhavan/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: invalid value encountered in log
  log_ttd = np.log(topic_term_dists)
/Users/mayamadhavan/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


LinAlgError: Array must not contain infs or NaNs